# Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from analysis.dim_reduction import Data
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

# Data

In [ ]:
# путь к папке с обработанными minian данными 
path_to_data = 'demo_movies'

# папки в вышеуказанной директории с обработанными minian различными записями и соотвутсвующие им группы
dates = {
    '25 october 2021': 'real',
    '28 october 2021': 'real',
    
    '8 november 2021': 'real',
    '11 november 2021': 'real',
    '15 november 2021': 'real',
    
    'artefact1': 'art_1', 
    'artefact2': 'art_1',
    'artefact3': 'art_1',
    
    'rb1': 'art_2',
    'rb2': 'art_2',
    'rb3': 'art_2',
    'rb4': 'art_2',
    'rb5': 'art_2',
    'rb6': 'art_2',
    'rb7': 'art_2',
    'rb8': 'art_2',
    'rb9': 'art_2',
    'rb10': 'art_2',
}

fps = 20

In [ ]:
%%time
# подгрузка данных
data_class = Data(path_to_data, dates.keys(), fps, dates.values())

In [ ]:
%%time
# сбор статистик
# операция занимает продолтительное время
df = data_class.get_data()

In [ ]:
# удалние сильноскоррелированных признаков
df = Data.drop_strong_corr(df)

In [ ]:
# снижение размерности
data, pca = Data.data_reduction(df)

In [ ]:
# визуализация результатов
Data.show_result(data, dates.values())

In [ ]:
# табличное представление результатов
result = pd.DataFrame({'x': data[:, 0],
                       'y': data[:, 1],
                       'date': dates.keys(),
                       'label': dates.values()
                      })
result

In [ ]:
# сохраниение результатов
result.to_csv('reduced_data.csv')

# PCA

In [ ]:
# веса признаков из PCA
feature_importance = pd.DataFrame(pca.components_, columns=df.columns)
feature_importance

In [ ]:
# центры масс по каждой категории записей
centers = result.pivot_table(values=['x', 'y'], index='label', aggfunc='mean')
centers

In [ ]:
# подсчет косинусного сходства
features = {'value': [],
             'label': [],
             'feature': [],
             'rating': []
            }
top_n = 8
for i, row in centers.iterrows():
    top = feature_importance.apply(lambda col: cosine_similarity([row], [col])[0,0])
    top = top[top.abs().sort_values(ascending=False).head(top_n).index]
    features['value'] += top.tolist()
    features['feature'] += top.index.tolist()
    features['label'] += [i] * (top_n)
    features['rating'] += list(np.arange(top_n))
    
features = pd.DataFrame(features)
features = features.pivot_table(values=['value', 'rating'], index=['label', 'feature'])
features = features.sort_values(by=['label', 'rating'])
features

In [ ]:
# визуализация наиболее выжных призанков по косинусному сходству
feature_df = features.reset_index()

fig, axs = plt.subplots(3, 1, figsize=(12, 15))
fig.subplots_adjust(hspace=.3)


for ax, label in zip(axs, feature_df['label'].unique()):
    
    sns.barplot(data = feature_df[feature_df['label'] == label], y='feature', x='value', ax=ax)
    ax.set_title(label, fontsize=18)
    
    ax.set_xlabel('value', fontsize=14)
    ax.set_ylabel('feature', fontsize=14)
    ax.tick_params(axis='both', labelsize=14)
    
plt.show()